# Analysis of "DOM_dedrift" files


Defining the variables:

In [ ]:
#Complete the dictionary with the values being the folders containing the datasets (each value is the path of the folder corresponding to one type of buffer and containing all the data associated to it). Note that the macro script should be run on the folder before using this code. The keys of the dictionary are the names you would like to give to the directories (label of the buffers).
paths = {}
# indicate the size of te pixels you want to compute the density (in nanometers)
length_nm = 0
#indicate the value of the first degree of freedom for each data
x_list = []
#indicate the value of the second degree of freedom for each data (in the same order)
y_list = []

Run this cell and then the cells of the codes you are interested in. Note that you can save the plots by uncommenting the last line of the cell you are considering.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize
import math
import scipy.stats
import statistics
from scipy.stats import gamma, lognorm, halflogistic, foldcauchy, exponnorm
from scipy.optimize import curve_fit
from scipy.stats import halfnorm
from scipy.interpolate import griddata
import scipy.integrate as integrate
import os
import re

number_paths = len(paths) 


l, l2, l3 = [], [], [] 
for path in list(paths.values()): 
    lp1 = os.listdir(path) 
    lp1 = list(filter(lambda x: x.endswith('avg.xls'), lp1))
    l2p1 = [os.path.join(path, f) for f in lp1]
    l.append(lp1)
    l3+=lp1
    l2+= l2p1    

d = dict(zip(l3, map(pd.read_table, l2)))

## Mean Density per microns squared

In [ ]:
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import numpy as np
import matplotlib.pyplot as plt

l, l2, l3 = [], [], [] 
for path in list(paths.values()): 
    lp1 = os.listdir(path) 
    lp1 = list(filter(lambda x: x.endswith('avg.xls'), lp1))
    l2p1 = [os.path.join(path, f) for f in lp1]
    l.append(lp1)
    l3+=lp1
    l2+= l2p1  
d = dict(zip(l3, map(pd.read_table, l2)))

mean_mole= {}
density= {}
for i in range(number_paths):
    j,m= 0,[]
    for file in l[i]:
        j+=1
        x_values = list(d[file]['X_(nm)'])
        y_values = list(d[file]['Y_(nm)'])
        bins_x = int(max([int(l) for l in x_values])/length_nm)
        bins_y = int(max([int(l) for l in y_values])/length_nm)
        returns = plt.hist2d(x_values, y_values, bins = (bins_x, bins_y), cmap ="gray")
        plt.title(list(paths.keys())[i]+": data "+str(j), fontweight ="bold")
        density[file] = [[int(bool(i)) for i in returns[0][j]] for j in range(bins_x)]
        m.append(len(d[file])/(sum(sum(i) for i in density[file])*(max(x_values)*0.001*max(y_values)*0.001/(bins_y*bins_x))))
    mean_mole[list(paths.keys())[i]] =np.mean(m)
plt.close()

print('Mean density of molecules per micron squared:', mean_mole);


## SNR

### SNR distribution per dataset

In [ ]:
idxfreIt = {}
for name, df in d.items():
    idxiter = [i for i in range(1,40)]
    idxfreIt[name] = (idxiter, [((df['SNR']> 0) &(df['SNR']>= idxiter[i]) &  (df['SNR'] < idxiter[i+1])).sum() for i in range(0, len(idxiter)-1) ])

    
m=0
for i in range(number_paths):
    j=0
    f = plt.figure()
    for name in l[i]:
        j+=1
        plt.bar(idxfreIt[name][0][:-1],np.array([idxfreIt[name][1][i]/len(d[list(d.keys())[m]]) for i in range(len(idxfreIt[name][1]))]), width = 0.6, label = "data "+ str(j))
        plt.title(list(paths.keys())[i])
        plt.xlabel("SNR")
        plt.ylabel("Fraction of Frames")
        plt.xlim([0,40])
        plt.legend()
        m+=1
    #uncomment next line to save the plot
    #plt.savefig(list(paths.keys())[i] +'SNR'+ '.png')

### Mean SNR distribution per dataset

In [ ]:
dxfreIt = {}
m=0
for i in range(number_paths):
    idxiter = [j for j in range(1,40)]
    idxfreIt[list(paths.keys())[i]] = (idxiter, [np.mean([((d[l[i][j]]['SNR']> 0) & (d[l[i][j]]['SNR']>= idxiter[k]) & (d[l[i][j]]['SNR'] < idxiter[k+1])).sum()/len(d[list(d.keys())[m+j]]) for j in range(len(l[i]))]) for k in range(0, len(idxiter)-1)])
    m+=len(l[i])
    
for i in range(number_paths):
    f = plt.figure()
    plt.bar(idxfreIt[list(paths.keys())[i]][0][:-1],height = np.array([idxfreIt[list(paths.keys())[i]][1][k] for k in range(len(idxfreIt[list(paths.keys())[i]][1]))]), width = 0.6, label = 'mean')
    plt.title(list(paths.keys())[i])
    plt.xlabel("SNR")
    plt.ylabel("Fraction of Frames")
    plt.xlim([0,40])
    plt.legend()
    #uncomment next line to save the plot
    #plt.savefig(list(paths.keys())[i] +'Average Distribution of the SNR'+'.png')

### Mean SNR per dataset

In [ ]:
la, l2a, l3a = [], [], [] 
for path in list(paths.values()): 
    lp1 = os.listdir(path) 
    lp1 = list(filter(lambda x: x.endswith('dedrift.xls'), lp1))
    l2p1 = [os.path.join(path, f) for f in lp1]
    la.append(lp1)
    l3a+=lp1
    l2a+= l2p1    

da = dict(zip(l3a, map(pd.read_table, l2a)))


SNR = {}
for i in range(number_paths):
    li=[]
    for name in l[i]:
        li.append(np.mean([d[name]['SNR'][s] for s in range(len(d[name]))]))
    SNR[list(paths.keys())[i]] = float("{:.2f}". format(np.mean(li)))

    
print('Mean SNR per dataset: ', SNR)


### Quantification of SNR decay per dataset

In [ ]:
idxfreIt = {}
m=0
for i in range(number_paths):
    idxiter = [j for j in range(1,40,5)]
    idxfreIt[list(paths.keys())[i]] = (idxiter, [np.mean([((d[l[i][j]]['SNR']> 0) & (d[l[i][j]]['SNR']>= idxiter[k]) & (d[l[i][j]]['SNR'] < idxiter[k+1])).sum()/len(d[list(d.keys())[m+j]]) for j in range(len(l[i]))]) for k in range(0, len(idxiter)-1)])
    m+=len(l[i])
print('Fraction of molecules with SNR between 11 and 16')
for i in range(number_paths):
    print(list(paths.keys())[i], np.array([idxfreIt[list(paths.keys())[i]][1][k] for k in range(len(idxfreIt[list(paths.keys())[i]][1]))])[idxfreIt[list(paths.keys())[i]][0][:-1].index(11)])
print('')

print('Fraction of molecules with SNR between 21 and 26')
for i in range(number_paths):
    print(list(paths.keys())[i], np.array([idxfreIt[list(paths.keys())[i]][1][k] for k in range(len(idxfreIt[list(paths.keys())[i]][1]))])[idxfreIt[list(paths.keys())[i]][0][:-1].index(21)])

### Density as a function of the SNR

In [ ]:
def getIndexes(table, value): 
    result = table.isin([value])
    return list(result['SNR'][result['SNR'] == True].index)

idx = {}
for name, df in d.items():
    idx[name] = (range(0, 70,5), [(df['SNR']>= i).sum() for i in range(0, 70,5)])

idxcnt = {}
for name, ids in idx.items():
    idxcnt[name] = (ids, [len(getIndexes(d[name], i)) for i in ids])
    
plt.figure() 
for i in range(number_paths):
        plt.plot(idx[l[i][0]][0], [np.mean([idx[name][1][j]/(sum(sum(i) for i in density[name])*(max(list(d[name]['X_(nm)']))*0.001*max(list(d[name]['Y_(nm)']))*0.001/(bins_y*bins_x))) for name in l[i]]) for j in range(len(idx[l[i][0]][0]))], '--', label = list(paths.keys())[i])
plt.legend()
plt.xlabel("SNR")
plt.xlim([0,50])
plt.ylabel("Density (molecules/microns^2)")
#uncomment next line to save the plot
#plt.savefig("All data DTT-sodium sulfite Density as a function of the SNR"+ '.png')

## Photon Count

### Photon Count distribution per dataset

In [ ]:

idxfreIt = {}
for name, df in d.items():
    idxiter = [i for i in range(1,math.ceil(max(df['IntegratedInt'])),500)]
    idxfreIt[name] = (idxiter, [((df['IntegratedInt']>= 0) & (df['IntegratedInt']>= idxiter[i]) & (df['IntegratedInt'] < idxiter[i+1])).sum() for i in range(0, len(idxiter)-1)])    

m=0
for i in range(number_paths):
    j=0
    f = plt.figure()
    for name in l[i]:
        j+=1
        plt.bar(idxfreIt[name][0][:-1],height = np.array([idxfreIt[name][1][k]/len(d[list(d.keys())[m]]) for k in range(len(idxfreIt[name][1]))]), width = 400, label = "data "+ str(j))
        plt.title(list(paths.keys())[i])
        plt.xlabel("Photon Count")
        plt.ylabel("Fraction of Frames")
        plt.xlim([0,6000])
        plt.legend()
        m+=1
    #uncomment next line to save the plot
    #plt.savefig(list(paths.keys())[i] +'Distribution of the photon count'+'.png')

### Mean Photon Count distribution per dataset

In [ ]:
idxfreIt = {}
m=0
for i in range(number_paths):
    idxiter = [j for j in range(1,6000,250)]
    idxfreIt[list(paths.keys())[i]] = (idxiter, [np.mean([((d[l[i][j]]['IntegratedInt']> 0) & (d[l[i][j]]['IntegratedInt']>= idxiter[k]) & (d[l[i][j]]['IntegratedInt'] < idxiter[k+1])).sum()/len(d[list(d.keys())[m+j]]) for j in range(len(l[i]))]) for k in range(0, len(idxiter)-1)])
    m+=len(l[i])

for i in range(number_paths):
    f = plt.figure()
    plt.bar(idxfreIt[list(paths.keys())[i]][0][:-1],height = np.array([idxfreIt[list(paths.keys())[i]][1][k] for k in range(len(idxfreIt[list(paths.keys())[i]][1]))]), width = 200, label = 'mean')
    plt.title(list(paths.keys())[i])
    plt.xlabel("Photon Count")
    plt.ylabel("Fraction of Frames")
    plt.xlim([-250,5000])
    plt.legend()  
    #uncomment next line to save the plot
    #plt.savefig(list(paths.keys())[i] +'Average Distribution of the photon count'+'.png')
    

### Mean photon count per dataset

In [ ]:
la, l2a, l3a = [], [], [] 
for path in list(paths.values()): 
    lp1 = os.listdir(path) 
    lp1 = list(filter(lambda x: x.endswith('dedrift.xls'), lp1))
    l2p1 = [os.path.join(path, f) for f in lp1]
    la.append(lp1)
    l3a+=lp1
    l2a+= l2p1    

da = dict(zip(l3a, map(pd.read_table, l2a)))

Integrated_int = {}
for i in range(number_paths):
    li=[]
    for name in l[i]:
        li.append(np.mean([d[name]['IntegratedInt'][s] for s in range(len(d[name])) if d[name]['IntegratedInt'][s]>0]))
    Integrated_int[list(paths.keys())[i]] = float("{:.2f}". format(np.mean(li)))
    

print('Integrated_int: ', Integrated_int)

### Quantification of the photon count decay per dataset

In [ ]:
idxfreIt = {}
m=0
for i in range(number_paths):
    idxiter = [j for j in range(0,6000,500)]
    idxfreIt[list(paths.keys())[i]] = (idxiter, [np.mean([((d[l[i][j]]['IntegratedInt']> 0) & (d[l[i][j]]['IntegratedInt']>= idxiter[k]) & (d[l[i][j]]['IntegratedInt'] < idxiter[k+1])).sum()/len(d[list(d.keys())[m+j]]) for j in range(len(l[i]))]) for k in range(0, len(idxiter)-1)])
    m+=len(l[i])
    
print('probability for photon count between 3000 and 3500')
for i in range(number_paths):
    print(list(paths.keys())[i], np.array([idxfreIt[list(paths.keys())[i]][1][k] for k in range(len(idxfreIt[list(paths.keys())[i]][1]))])[idxfreIt[list(paths.keys())[i]][0][:-1].index(3000)])
print('')

print('probability for photon count between 4000 and 4500')
for i in range(number_paths):
    print(list(paths.keys())[i], np.array([idxfreIt[list(paths.keys())[i]][1][k] for k in range(len(idxfreIt[list(paths.keys())[i]][1]))])[idxfreIt[list(paths.keys())[i]][0][:-1].index(4000)])

## 2D histograms

### 2D histogram for the SNR

In [ ]:
x = x_list
y = y_list
SNR_val = list(SNR.values())
x_grid = np.arange(min(x_list)-5,max(x_list)+5,1)#rechange as needed 
y_grid = np.arange(min(y_list)-5,max(y_list)+5,1)#rechange as needed 
[x_grid,y_grid] = np.meshgrid(x_grid,y_grid)
z_grid = griddata((x,y), SNR_val, (x_grid, y_grid))
fig = plt.figure()
ax = fig.add_subplot(111)
plt.contourf(x_grid,y_grid,z_grid)
plt.plot(x,y, 'k.')
plt.xlabel('DTT')#rechange as needed 
plt.ylabel('Sodium Sulfite')#rechange as needed 
cbar = plt.colorbar()
cbar.ax.set_ylabel('SNR')
#uncomment next line to save the plot
#plt.savefig(list(paths.keys())[i] +'SNR 2D histogram DTT-SS'+ '.png')


### 2D histogram for the photon count

In [ ]:
x = x_list
y = y_list
Int_val = list(mean_mole.values())
x_grid = np.arange(min(x_list)-5,max(x_list)+5,1)#rechange as needed 
y_grid = np.arange(min(y_list)-5,max(y_list)+5,1)#rechange as needed 
[x_grid,y_grid] = np.meshgrid(x_grid,y_grid)
z_grid = griddata((x,y), Int_val, (x_grid, y_grid))
fig = plt.figure()
ax = fig.add_subplot(111)
plt.contourf(x_grid,y_grid,z_grid)
plt.plot(x,y, 'k.')
plt.xlabel('DTT')#rechange as needed 
plt.ylabel('Sodium Sulfite')#rechange as needed 
cbar = plt.colorbar()
cbar.ax.set_ylabel('Density')
#uncomment next line to save the plot
#plt.savefig(list(paths.keys())[i] +'Density 2D histogram DTT-SS'+ '.png')